## Multi Linear regression 적용
- Train 데이터로 학습된 xgboost로 Test 데이터 평가 -> Fully - Paid라고 확신하는 데이터 남겨둔다.
- Train 데이터로 학습된 regression 모델로 Fully - Paid라고 확신한 데이터 regression 진행
- 상위 10% 데이터를 이용하여 portfolio를 만든다.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print(device)

cuda


In [7]:
## Load original data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/train_dataset.csv"
data = pd.read_csv(data_path, low_memory=False)

##load test data
test_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/test_dataset.csv"
test_data = pd.read_csv(test_data_path)

return_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/return_feature_test.csv"
return_data = pd.read_csv(return_data_path)

In [10]:
return_data.head()

,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,zip_code,installment,fico_range_low,total_rev_hi_lim,avg_cur_bal,bc_open_to_buy,revol_util,revol_bal,total_acc,total_pymnt,total_pymnt_inv,loan_amnt,credit_history_years,loan_status
0,504.0,142.0,0.1335,140000.0,7.07,941.0,253.98,670.0,9000.0,10538.0,2946.0,0.673,6054.0,8.0,9142.877884,9142.88,7500.0,12,0
1,689.0,79.0,0.2450,37000.0,34.38,349.0,290.59,690.0,16000.0,9859.0,4211.0,0.568,9089.0,24.0,12570.430000,12570.43,10000.0,6,0
2,739.0,192.0,0.0999,72000.0,10.35,480.0,322.63,680.0,18800.0,10397.0,705.0,0.699,13147.0,22.0,10330.460000,10330.46,10000.0,16,0
3,739.0,236.0,0.0721,72000.0,32.13,481.0,371.68,800.0,140200.0,5541.0,78841.0,0.066,9277.0,50.0,12817.155956,12817.16,12000.0,20,0
4,709.0,155.0,0.0762,90000.0,19.29,283.0,405.10,690.0,53700.0,8023.0,32441.0,0.325,17459.0,46.0,14583.506490,14583.51,13000.0,13,0


In [4]:
data_x = data.drop(columns='loan_status')
data_y = data[['loan_status']]

X_test = test_data.drop(columns='loan_status')
y_test = test_data[['loan_status']]

In [5]:
# data under 6:2:2 train,val,test set
X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.33, random_state=42, stratify=data_y)

print(f"Train size: {X_train.shape[0]} samples ({X_train.shape[0]/len(data):.1%})")
print(f"Validation size: {X_val.shape[0]} samples ({X_val.shape[0]/len(data):.1%})")
print(f"Test size: {X_test.shape[0]} samples ({X_test.shape[0]/len(data):.1%})")

Train size: 997369 samples (67.0%)
Validation size: 491242 samples (33.0%)
Test size: 372153 samples (25.0%)
